In [4]:
from tkinter import *
from tkinter import ttk
import numpy as np
from scipy.linalg import svd
from tkinter import filedialog
from PIL import ImageTk, Image
import cv2 as cv2

In [5]:
def chooseImgs():
    imgPoints1.clear()
    imgPoints2.clear()
    epipolarMode.set(False)
    modeChk.state(['disabled'])
    
    #Choose the left image
    global img1Url
    img1Url = filedialog.askopenfilename(filetypes=(("jpg files","*.jpg"),("all files","*.*")))
    canvas1.img = ImageTk.PhotoImage(Image.open(img1Url).resize((320,240)))
    canvas1.create_image(0, 0, image=canvas1.img, anchor="nw")

    #choose the right image
    global img2Url
    img2Url = filedialog.askopenfilename(filetypes=(("jpg files","*.jpg"),("all files","*.*")))
    canvas2.img = ImageTk.PhotoImage(Image.open(img2Url).resize((320,240)))
    canvas2.create_image(0, 0, image=canvas2.img, anchor="nw")

In [23]:
def handlerImg1(event):
    if len(imgPoints1)<10:
        x,y = event.x,event.y
#         print("first: ",x,y)
        imgPoints1.append([x,y])
        canvas1.create_oval(x-2.5,y-2.5,x+2.5,y+2.5,fill=dot_color, tags='dots')
    # print("img1:",imgPoints1)
#     if len(imgPoints1) == 10 and len(imgPoints2) == 10:
#         epiModeChk.state(['!disabled'])
        
    if epipolarMode.get():
        x,y = event.x,event.y
#         plot_epipolar_line(matrixF)
        canvas1.create_oval(x-2.5,y-2.5,x+2.5,y+2.5,fill=dot_color, tags='dots')
        plot_epipolar_line(x, y)
    
def handlerImg2(event):
    if len(imgPoints2)<10:
        x,y = event.x,event.y
        # print("second: ",x,y)
        imgPoints2.append([x,y])
        canvas2.create_oval(x-2.5,y-2.5,x+2.5,y+2.5,fill=dot_color, tags='dots')
    # print("img2:",imgPoints2)
    if len(imgPoints1) == 10 and len(imgPoints2) == 10:
        modeChk.state(['!disabled'])

In [7]:
def computeFundamentalMatrix():
    print("mode chck:",epipolarMode.get())
    
    global matrixF
    
    if epipolarMode.get():
        canvas1.delete('dots')
        canvas2.delete('dots')
        
        matrixF, mask = cv2.findFundamentalMat(np.array(imgPoints1),np.array(imgPoints2),cv2.FM_LMEDS)
        
        print(matrixF)

In [8]:
def computeEpipolarPoints(u,v):
    # print(matrixF)
    f1 = matrixF[0]
    f2 = matrixF[1]
    f3 = matrixF[2]
    v1 = -(0*(u*f1[0]+v*f1[1]+f1[2])+u*f3[0]+v*f3[1]+f3[2])/(u*f2[0]+v*f2[1]+f2[2])
    v2 = -(1080*(u*f1[0]+v*f1[1]+f1[2])+u*f3[0]+v*f3[1]+f3[2])/(u*f2[0]+v*f2[1]+f2[2])
    # u1 = -(0*(u*f2[0]+v*f2[1]+f2[2])+u*f3[0]+v*f3[1]+f3[2])/(u*f1[0]+v*f1[1]+f1[2])
    # u2 = -(1080*(u*f2[0]+v*f2[1]+f2[2])+u*f3[0]+v*f3[1]+f3[2])/(u*f1[0]+v*f1[1]+f1[2])
    # return int(u1),0,int(u2),1080
    return 0,v1,1080,v2

In [9]:
def getSearchRange(u,v,n):
    # print(matrixF)
    f1 = matrixF[0]
    f2 = matrixF[1]
    f3 = matrixF[2]
    v1 = -((u-n)*(u*f1[0]+v*f1[1]+f1[2])+u*f3[0]+v*f3[1]+f3[2])/(u*f2[0]+v*f2[1]+f2[2])
    v2 = -((u+n)*(u*f1[0]+v*f1[1]+f1[2])+u*f3[0]+v*f3[1]+f3[2])/(u*f2[0]+v*f2[1]+f2[2])
    return u-n,v1,u+n,v2

In [10]:
def computeAvg(img,u,v,n):
    sum = 0
    for i in range(-n,n+1):
        for j in range(-n,n+1):
            if((u+i)<frameWidth and (v+j)<frameHeight):
                # print((u+i),(v+j))
                sum+=img[u+i][v+j]
    return float(sum)/(2*n+1)**2

In [11]:
def computeStdDeviation(img,u,v,n):
    sum = 0
    avg = computeAvg(img,u,v,n)
    for i in range(-n,n+1):
        for j in range(-n,n+1):
            if((u+i)<frameWidth and (v+j)<frameHeight):
                sum+= (img[u+i][v+j] - avg)**2
    return (sum**0.5)/(2*n+1)

In [12]:
def ZNCC(img1,img2,u,v,u1,v1,n):
    std_deviation1 = computeStdDeviation(img1,u,v,n)
    std_deviation2 = computeStdDeviation(img2,u1,v1,n)
    avg1=computeAvg(img1,u,v,n)
    avg2=computeAvg(img2,u1,v1,n)
    # print(std_deviation1,std_deviation2)
    sum=0
    for i in range(-n,n+1):
        for j in range(-n,n+1):
            if((u+i)<frameWidth and (v+j)<frameHeight and (u1+i)<frameWidth and (v1+j)<frameHeight):
                sum+=(img1[u+i][v+j] - avg1)*(img2[u1+i][v1+j] - avg2)
            
    return float(sum)/((2*n+1)**2 * std_deviation1*std_deviation2)

In [28]:
def plot_epipolar_line(u,v):

    u1,v1,u2,v2 = computeEpipolarPoints(u,v)
    # print(u1,v1,u2,v2)
    canvas2.create_line(u1,v1,u2,v2,fill='blue',width=2)
    x1,y1,x2,y2 = getSearchRange(u,v,50)
    # print(x1,y1,x2,y2)
    # canvas2.create_line(x1,y1,x2,y2,fill='red',width=5)
    # p1 = np.array([u1,v1])
    # p2 = np.array([u2,v2])
    im1= Image.open(img1Url).resize((640,480))
    im2= Image.open(img2Url).resize((640,480))
    img1 = np.asarray(im1)
    img2 = np.asarray(im2)
    points = getLinePoints(x1,y1,x2,y2)
    maxZNCC = 0.0
    matchedPixel = (0,0)
    for p in points:
#         print(p)
#         print(ZNCC(img1,img2,int(u),int(v),int(p[0]),int(p[1]),50))
#         canvas2.create_text(p[0],p[1],fill='red',font="Times 12",text="X")
        zncc = ZNCC(img1,img2,int(u),int(v),int(p[0]),int(p[1]),5)
        if(maxZNCC<zncc):
            maxZNCC = zncc
            matchedPixel = p
    canvas2.create_text(matchedPixel[0],matchedPixel[1],fill='yellow',font="Times 8",text="O")

In [20]:
def getLinePoints(x1, y1, x2, y2):
    points = []
    issteep = abs(y2-y1) > abs(x2-x1)
    if issteep:
        x1, y1 = y1, x1
        x2, y2 = y2, x2
    rev = False
    if x1 > x2:
        x1, x2 = x2, x1
        y1, y2 = y2, y1
        rev = True
    deltax = x2 - x1
    deltay = abs(y2-y1)
    error = int(deltax / 2.0)
    y = y1
#     ystep = None
    if y1 < y2:
        ystep = 1
    else:
        ystep = -1
    for x in np.arange(x1, x2 + 1,1):
        coord = (y, x) if issteep else (x, y)
        points.append(coord)
        error -= abs(deltay)
        if error < 0:
            y += ystep
            error += deltax
#            

        
    # Reverse the list if the coordinates were reversed
#     if rev:
#         points.reverse()
    return points

In [29]:
if __name__ == "__main__":
    frameWidth = 1080
    frameHeight = 1080
    imgPoints1 = []
    imgPoints2 = []
    matrixA = []

    dot_color = "red"
    main_window = Tk()
    main_window.title("COMP 8510 - Project 3")

    button = ttk.Button(main_window, text = "Choose Left and right images", command = chooseImgs)
    button.grid(row = 0 ,column = 0)

    #display first image
    frame1 = ttk.Frame(main_window)
    frame1.grid(row=1,column=0)
    frame1.config(relief=SOLID)
    
    #display second image
    frame2 = ttk.Frame(main_window)
    frame2.grid(row=1,column=1)
    frame2.config(relief=SOLID)

    #10 points on the first image
    canvas1 = Canvas(frame1, width=320,height=240)
    canvas1.bind('<Button-1>',handlerImg1)
    canvas1.pack()
    
    #10 points in the second image
    canvas2 = Canvas(frame2, width=320,height=240)
    canvas2.bind('<Button-1>',handlerImg2)
    canvas2.pack()

    #Compute Fundamental Matrix
    frame3 = ttk.Frame(main_window)
    frame3.grid(row=2,column=0)
    frame3.config()

    epipolarMode = BooleanVar()
    modeChk = ttk.Checkbutton(frame3, text="Epiploar Mode", variable=epipolarMode)
    modeChk.config(command = computeFundamentalMatrix)

    #Stop for 10 points
    modeChk.pack() 
#     if len(imgPoints1) < 10 or len(imgPoints2) < 10:
#         modeChk.state(['disabled'])
    frame4 = ttk.Frame(main_window)
    frame4.grid(row=2,column=1)
    frame4.config()

    pixelMode = BooleanVar()
    pixModeChk = ttk.Checkbutton(frame4, text="Pixel Matching Mode", variable=pixelMode)
    # pixModeChk.config(command = )
    # pixModeChk.pack()
    # matrixF = [[ 1.58406946e-07,  4.51762381e-07,  6.22376948e-03], [-2.69211010e-06, -2.24845556e-07, -6.57431845e-02], [-7.95642642e-03,  6.66038165e-02,  9.95560006e-01]]
#     matrixF = [[ 1.60230126e-06, -1.20558896e-05,  1.70701100e-02], [ 8.78526657e-06, -6.92767089e-07, -7.79206768e-02], [-1.98507659e-02,  7.98544041e-02,  9.93411395e-01]]

    main_window.mainloop()

mode chck: True
[[-6.19385038e-04  9.95688479e-03 -5.96506397e-01]
 [-1.01035210e-02  8.49574829e-04  9.98010951e-02]
 [ 6.93429394e-01 -1.87325167e-01  1.00000000e+00]]
